А точно нужно сортировать по вероятности? <br>
Сортировать нужно, т.к. от этого будет зависеть порядок рекомендаций товаров в емэйл. Необходимо максимально заинтересовать покупателя первым товаром. В эмейле он должен быть либо самым левым, либо самым верхним

Какую метрику использовать? <br>
Метрику желательно использовать, учитывающую порядок рекомендаций. Это может быть, например, MAP@k еслм нам нужна общая метрика по всем пользователям

Сколько раз в неделю отпрпавляем рассылку?  <br>
Если это продуктовый ритейл, то можно рассылку делать два раза в неделю. Исходя из того, что большинство людей посещают супермаркеты один-два раза в неделю. 
В идеале, можно персониализировать частоту рассылки для пользоветей, отслеживая их посещение магазинов и производить рассылку за 1-2 дня до их предполагаемых посещений.

В какое время отправляем рассылку?  <br>
Лучше отправлять рассылку в первой половине дня, чтобы большая часть покупателей ознакомилась с рассылкой в течение дня и посетила магазин после работы если их заинтересовали наши рекомендации. 

Будем отправлять одному юзеру много раз наши рекоммендации. Как добиться того, чтобы они хоть немного отличались?  <br>
Можно разбавлять наши рекомендации товарами, похожими на рекомендованные, можно добавлять в рекомендации акционные товары из товарных категорий, которые популярны у пользователя, но которые не попали в рекомендации.

Нужно ли, чтобы в одной рассылке были разные товары? Как определить, что товары разные? Как добиться того, чтобы они были разными?  <br>
Всё зависит от предпочтений пользоветаля. Если он каждый раз покупает одни и те же, товары, маловероятно, что его заинтересуют товары, сильно отличающиеся от его корзины.  <br>
Если корзина пользователя более вариантна, рассылка для него должна быть с разными товарами. Опеределить похожесть товаров можно, например, рассчитав косинусное расстояние (или другое) между товарами
в товарной матрице.


In [52]:
import mkl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    

In [53]:
mkl.set_num_threads(1)

1

In [54]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision


def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [55]:
data = pd.read_csv('./data/retail_train.csv')

test_size_weeks = 3
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]
data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [56]:
item_features = pd.read_csv('./data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)


In [57]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [58]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [59]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [61]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [62]:
# %%time
    
# result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
# print(result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean())

In [63]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [65]:
from collections import defaultdict
from tqdm.notebook import tqdm
best_params = defaultdict(list)
factors = [32, 64, 96, 128] #, 160, 192]
regularizations = [0.001, 0.01, 1.] #, 10.]
iterations = [10, 20, 30, 40]

for facts in tqdm(factors, desc="factors"):
    for regularization in tqdm(regularizations, desc="regularizations", leave=False):
        for iters in tqdm(iterations, desc="iterations", leave=False):
            model = AlternatingLeastSquares(factors=facts, 
                                regularization=regularization,
                                iterations=iters, 
                                calculate_training_loss=True, 
                                num_threads=4, use_gpu=False) # K - кол-во билжайших соседей

            model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=False)

            result['als_bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

            res = result.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean()
            best_params['n_factors'].append(facts)
            best_params['regularization'].append(regularization)
            best_params['iterations'].append(iters)
            best_params['precision_at_k_mean'].append(res)

df = pd.DataFrame(best_params)
df.sort_values('precision_at_k_mean', ascending=False, inplace=True)  
df   

factors:   0%|          | 0/4 [00:00<?, ?it/s]

regularizations:   0%|          | 0/3 [00:00<?, ?it/s]

iterations:   0%|          | 0/4 [00:00<?, ?it/s]

iterations:   0%|          | 0/4 [00:00<?, ?it/s]

iterations:   0%|          | 0/4 [00:00<?, ?it/s]

...

regularizations:   0%|          | 0/3 [00:00<?, ?it/s]

iterations:   0%|          | 0/4 [00:00<?, ?it/s]

iterations:   0%|          | 0/4 [00:00<?, ?it/s]

iterations:   0%|          | 0/4 [00:00<?, ?it/s]

,n_factors,regularization,iterations,precision_at_k_mean
40,128,0.010,10,0.200686
36,128,0.001,10,0.199412
41,128,0.010,20,0.198041
42,128,0.010,30,0.197356
38,128,0.001,30,0.195397
39,128,0.001,40,0.195201
37,128,0.001,20,0.194515
43,128,0.010,40,0.194515
45,128,1.000,20,0.194221
44,128,1.000,10,0.194221
